## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

## Clean Data

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [8]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon
1,Apogee South Beach\r800 S Pointe Dr 1201\rMiam...,4/3/2023,234,"$16,200,000.00",6595,"$3,899.86",Jill Hertzberg,Coldwell Banker Realty,Gerrit Kocks,OWN Realty,Apogee South Beach 800 S Pointe Dr 1201 Miami ...,Apogee South Beach,800 S Pointe Dr 1201 Miami Beach,"(25.7677245, -80.13634490000001)",25.767724,-80.136345
2,Estates at Acqualna\r17901 Collins Ave 1605\rS...,4/3/2023,126,"$11,625,000.00",5222,"$2,226.16",Sandra Pariente,Charles Rutenberg Realty Fort,JTHS Accounting,MIAMI Association of REALTORS®,Estates at Acqualna 17901 Collins Ave 1605 Sun...,Estates at Acqualna,17901 Collins Ave 1605 Sunny Isles Beach,"(25.9420203, -80.1201787)",25.942020,-80.120179
3,Grovenor House\r2677 S Bayshore Dr 2801\rMiami,4/3/2023,12,"$5,475,000.00",4370,"$1,252.86",Judith Zeder,Coldwell Banker Realty,Blair Sonville,Lowell International Rlty LLC,Grovenor House 2677 S Bayshore Dr 2801 Miami,Grovenor House,2677 S Bayshore Dr 2801 Miami,"(25.7322307, -80.2348947)",25.732231,-80.234895
4,Isles of Normandy\r1930 Bay Dr PH-1\rMiami Beach,4/4/2023,56,"$4,800,000.00",2551,"$1,881.62",Dora Puig,Luxe Living Realty,Dora Puig,Luxe Living Realty,Isles of Normandy 1930 Bay Dr PH-1 Miami Beach,Isles of Normandy,1930 Bay Dr PH-1 Miami Beach,"(25.8528739, -80.13504549999999)",25.852874,-80.135045
5,Jade Ocean\r17121 Collins Ave 4404\rSunny Isle...,4/3/2023,153,"$4,000,000.00",4303,$929.58,Felise Eber,Coldwell Banker Realty,Brian Bartley,"Brickell Realty Group, LLC",Jade Ocean 17121 Collins Ave 4404 Sunny Isles ...,Jade Ocean,17121 Collins Ave 4404 Sunny Isles Beach,"(25.9344014, -80.1210054)",25.934401,-80.121005
6,Jade Brickell\r1331 Brickell Bay Dr 3001\rMiami,4/3/2023,256,"$3,600,000.00",3415,"$1,054.17",Andres Leiser,"Constec Realty Services, Inc",Omar Morales,Outlook Realty,Jade Brickell 1331 Brickell Bay Dr 3001 Miami,Jade Brickell,1331 Brickell Bay Dr 3001 Miami,"(25.7603442, -80.1897888)",25.760344,-80.189789
7,Casa Del Mar\r881 Ocean Dr 21H\rKey Biscayne,4/6/2023,43,"$3,385,000.00",3120,"$1,084.94",Jacqueline Cabrera,One Sotheby's International Realt,George Formento,"Compass Florida, LLC.",Casa Del Mar 881 Ocean Dr 21H Key Biscayne,Casa Del Mar,881 Ocean Dr 21H Key Biscayne,"(25.6857044, -80.15754919999999)",25.685704,-80.157549
8,Brickell Flatiron\r1000 Brickell Plaza PH5001\...,4/4/2023,185,"$3,050,000.00",1955,"$1,560.10",Lilian Carrion,Charles Rutenberg Realty FTL,Nelson Gonzalez PA,BHHS EWM Realty,Brickell Flatiron 1000 Brickell Plaza PH5001 M...,Brickell Flatiron,1000 Brickell Plaza PH5001 Miami,"(25.7642427, -80.1927022)",25.764243,-80.192702
9,Atlantic One at the Point\r21200 NE 38th Ave 2...,4/6/2023,3,"$2,350,000.00",4764,$493.28,Dallas Wharton,Inkwell Realty LLC,Dallas Wharton,Inkwell Realty LLC,Atlantic One at the Point 21200 NE 38th Ave 23...,Atlantic One at the Point,21200 NE 38th Ave 2305 & 311 Aventura,"(25.9730612, -80.12504779999999)",25.973061,-80.125048
10,0Ocean Club\r791 Crandon Blvd 202\rKey Biscayne,4/3/2023,87,"$2,350,000.00",2510,$936.26,Gloria Ramirez,Great Properties Int'l LLC,Nicholas Richberg,Realty Hub,0Ocean Club 791 Crandon Blvd 202 Key Biscayne,,0Ocean Club 791 Crandon Blvd 202 Key Biscayne,"(25.688277, -80.158928)",25.688277,-80.158928


In [9]:
df.at[10,'building_name']=('Ocean Club')
df.at[10,'address_only']=('791 Crandon Blvd')

## Format Data

In [10]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [11]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=True)
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=True)
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=True)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=True)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [12]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [13]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [14]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [15]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('April 2nd - April 8th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [16]:
m.save('index.html')

## Data snagger

In [17]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [18]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Apogee South Beach , 800 S Pointe Dr 1201 Miami Beach | Price $16,200,000 | $3,900 psf | Listing agent: Jill Hertzberg with Coldwell Banker Realty | Buyer's agent: Gerrit Kocks with OWN Realty | Days on market: 234
Least Expensive
Atlantic One at the Point , 21200 NE 38th Ave 2305 & 311 Aventura | Price $2,350,000 | $493 psf | Listing agent: Dallas Wharton with Inkwell Realty LLC | Buyer's agent: Dallas Wharton with Inkwell Realty LLC | Days on market: 3
Highest Price Per Square Foot
Apogee South Beach , 800 S Pointe Dr 1201 Miami Beach | Price $16,200,000 | $3,900 psf | Listing agent: Jill Hertzberg with Coldwell Banker Realty | Buyer's agent: Gerrit Kocks with OWN Realty | Days on market: 234
Lowest Price Per Square Foot
Atlantic One at the Point , 21200 NE 38th Ave 2305 & 311 Aventura | Price $2,350,000 | $493 psf | Listing agent: Dallas Wharton with Inkwell Realty LLC | Buyer's agent: Dallas Wharton with Inkwell Realty LLC | Days on market: 3
Most Days on Market
Jade

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [19]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [20]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_04102023
